<a id="part0"></a>
# 0.Intro:

![](https://upload.wikimedia.org/wikipedia/commons/1/1a/MS_Estonia_model.jpg)

What i did at this notebook let me summarize it for you. Dataset has 9 columns and 989 rows. 3 column("PassengerId","Firstname","Lastname") needless for training so i dropped these columns at the begining.After dropping i check nan values. Lucky for me dataset has 0 nan values :). I drew 8 plot to understand column releationships.It looks like 0-20 age range ,crew and males has more chance to survive from disaster.To analyze data i have to transform all categorical data to numerical data.To do that i applied OneHotEncode and Label Encode to 3 column. At disaster %80 of passengers died. That means we have unbalanced dataset here. I used smote for balancing.I didn't tune any hyperparameter but i compared 4 different classification method success rates.

**Table of Contents **
* [0.Intro:](#part0)
* [1.Deleting unnecessary columns:](#part1)
* [2.Exploratory Data Analysis:](#part2)
* [3.Encoding:](#part3)
* [4.Traning:](#part4)
* [5.Data Augmentation:](#part5)
* [6.Training Again:](#part6)
* [7.Evaluation:](#part7)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data = pd.read_csv("/kaggle/input/passenger-list-for-the-estonia-ferry-disaster/estonia-passenger-list.csv")

In [ ]:
data.head(),data.shape

We have 989 rows and 8 columns.But we are going to drop passengerId , firstname and lastname columns because we dont need these for training.

In [ ]:
data.Survived.value_counts()

Lets check null values

In [ ]:
data.isnull().sum()

No missing values. Perfect!

<a id="part1"></a>
# 1.Deleting unnecessary columns:

We don't need PassengerId , Firstname and Lastname columns for training so we can drop this columns.

In [ ]:
data.drop(["PassengerId","Firstname","Lastname"],axis = 1,inplace = True)

<a id="part2"></a>
## 2.Exploratory Data Analysis: 

In [ ]:
plt.figure(figsize=(15,10))
sns.barplot(x = data.Country.unique(),y = data.Country.value_counts()) , plt.title("Passenger count by country")

Most of our passengers from Sweden and Estonia.

In [ ]:
plt.figure(figsize=(15,10))
sns.barplot(x = data.Sex.unique(),y = data[data.Survived == 1]["Sex"].value_counts() * 100 / data.Sex.value_counts()) , plt.title("survival percentage by gender")

Seems like males survival rate 4 times bigger than females

In [ ]:
plt.figure(figsize=(15,10))
sns.barplot(x = list(np.sort(data[data.Survived == 1]["Country"].unique())),y = data[data["Survived"] == 1]["Country"].value_counts().sort_index())
plt.title("How many passengers survived by country")

Estonian and Swedish passengers survived most. Because most of our passengers from Sweden and Estonia.

In [ ]:
pd.cut(data[data["Survived"] == 1]["Age"],bins= 8, labels = ["0-10","11-21","22-32","33-43","44-54","55-65","66-76","76-90"]).value_counts().sort_index()

In [ ]:
fig , axs = plt.subplots(1,2,figsize = (15,8))
sns.distplot(data.Age , bins = 8, ax = axs[0]).set_title('Passengers age distribution')
sns.barplot(x = ["0-10","11-21","22-32","33-43","44-54","55-65","66-76","76-90"], y= pd.cut(data[data["Survived"] == 1]["Age"],bins= 8, labels = ["0-10","11-21","22-32","33-43","44-54","55-65","66-76","76-90"]).value_counts().sort_index()
            * 100 / pd.cut(data.Age,bins= 8, labels = ["0-10","11-21","22-32","33-43","44-54","55-65","66-76","76-90"]).value_counts().sort_index(), ax = axs[1]).set_title('Passengers age distribution')
plt.title("Survive percentage by age." )
data.Age.mean()

Mean value of passengers age is 44. We have less passengers at 0-20 and +70 age range also survive rate is higher at 0-20 and 76-90 age range.

In [ ]:
fig , axss = plt.subplots(1,2,figsize = (15,8))
sns.barplot(data.Category.unique(),data[data["Survived"] == 1]["Category"].value_counts() *100 / data.Category.value_counts(),ax = axss[0]).set_title("Survive percentage by category.(C=Crew, P=Passenger)")
sns.barplot(data.Category.unique(),data.Category.value_counts(),ax = axss[1]).set_title("Passenger and Crew count")

Crew members have more chance to survive than passengers but still its around 20%

<a id="part3"></a>
# 3.Encoding:

I'am going to label encode "Sex" and "Category" columns because these columns has 2 unique values.But "Country" column has unique values more than 2 and Country value is nominal data.We are going to apply OneHotEncoding to Country column.

In [ ]:
data.head()

In [ ]:
le = LabelEncoder()
ohe = OneHotEncoder(sparse = False)
data.Sex = le.fit_transform(data.Sex) # Label Encoding
data.Category = le.fit_transform(data.Category)
data = pd.concat((data ,pd.DataFrame(ohe.fit_transform(data.Country.to_frame()),columns = ohe.get_feature_names())),axis = 1) #I concatenate the data and one hot encoded column here.
data.drop(["Country"],inplace = True , axis = 1) # After concatenation i dropped country column.

Lets check the dataset

In [ ]:
data.head()

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(data.corr() , annot = True) # Correlation matrix

<a id="part4"></a>
# 4.Traning:

Before training lets split the dataset first.

In [ ]:
X = data.drop(columns = ["Survived"])
y = data.Survived
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state=10)

![](https://miro.medium.com/max/712/1*Z54JgbS4DUwWSknhDCvNTQ.png)
* Sensitivity - True Positive Rate = TP / (TP + FN)
* Specificity - True Negative Rate = TN / (TN + FP)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_predlr = lr.predict(X_test)
cmlr = confusion_matrix(y_test , y_predlr)
print(f"Accuracy: {accuracy_score(y_test, y_predlr)} Sensitivity: {cmlr[0,0] / (cmlr[0,0]+cmlr[1,0])} Specificity: {cmlr[1,1] / (cmlr[1,1]+cmlr[0,1])}")

Accually model predict very bad. Specificity 0 means we predicted nothing at survived = 1. This is happening because of imbalanced dataset.lets balance data.

<a id="part5"></a>
# 5.Data Augmentation:

We are going to use Smote for over-sampling.First we have to download imblearn library

In [ ]:
pip install imblearn

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE()
X_sm, y_sm = sm.fit_resample(X, y)
y.value_counts() , y_sm.value_counts()

Before we had 137 survivors. Now we have 852. Perfect!Lets train again and evaluate results.

<a id="part6"></a>
# 6.Training Again:

Before training we have to split data again

In [ ]:
X_train_sm, X_test_sm, y_train_sm, y_test_sm = train_test_split(X_sm,y_sm,test_size = 0.2, random_state=10)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train_sm, y_train_sm)
y_predlr2 = lr.predict(X_test_sm)
cmlr2 = confusion_matrix(y_test_sm , y_predlr2)
print(f"Accuracy: {accuracy_score(y_test_sm, y_predlr2)} Sensitivity: {cmlr2[0,0] / (cmlr2[0,0]+cmlr2[1,0])} Specificity: {cmlr2[1,1] / (cmlr2[1,1]+cmlr2[0,1])}")

Our accuracy decreased %15 but as you can see now we started to predict 1 values from y variable.The variable y was filled with zeros before smote.Now we can try different classifiers.

In [ ]:
svc = SVC() #Assignment
rf = RandomForestClassifier()
knn = KNeighborsClassifier()

In [ ]:
svc.fit(X_train_sm, y_train_sm) # Traning
rf.fit(X_train_sm, y_train_sm)
knn.fit(X_train_sm, y_train_sm)

In [ ]:
y_predsvc = svc.predict(X_test_sm) #Prediction
y_predrf = rf.predict(X_test_sm)
y_predknn = knn.predict(X_test_sm)

In [ ]:
cmsvc = confusion_matrix(y_test_sm , y_predsvc) #Confusion Matrix
cmrf = confusion_matrix(y_test_sm , y_predrf)
cmknn = confusion_matrix(y_test_sm , y_predknn)

<a id="part7"></a>
# 7.Evaluation:

In [ ]:
print(f"Accuracy of Logistic Regression: {accuracy_score(y_test_sm, y_predlr2)} Sensitivity: {cmlr2[0,0] / (cmlr2[0,0]+cmlr2[1,0])} Specificity: {cmlr2[1,1] / (cmlr2[1,1]+cmlr2[0,1])}")
print(f"Accuracy of Support Vector Machine Classifier: {accuracy_score(y_test_sm, y_predsvc)} Sensitivity: {cmsvc[0,0] / (cmsvc[0,0]+cmsvc[1,0])} Specificity: {cmsvc[1,1] / (cmsvc[1,1]+cmsvc[0,1])}")
print(f"Accuracy of Random Forest Classifier: {accuracy_score(y_test_sm, y_predrf)} Sensitivity: {cmrf[0,0] / (cmrf[0,0]+cmrf[1,0])} Specificity: {cmrf[1,1] / (cmrf[1,1]+cmrf[0,1])}")
print(f"Accuracy of KNeighbor Classifier: {accuracy_score(y_test_sm, y_predknn)} Sensitivity: {cmknn[0,0] / (cmknn[0,0]+cmknn[1,0])} Specificity: {cmknn[1,1] / (cmknn[1,1]+cmknn[0,1])}")

Without any hyperparameter tuning random forest classifier gave us best results. Thanks for your time. What you think about dataset and my analyze? If you have any questions or suggestions let me know. Have a nice day.